In [1]:
import numpy as np
x = np.load('data/dataset.npy')
idx_to_word = {}
word_to_idx = {}
def _process_vocabs(data): 
        """
        process all the words and return the idx of each word in the sentence
        """
        all_words = []
        for i in data:
            all_words = all_words+i
        #list of all words in the dataset    
        vocab = sorted(list(set(all_words)))
        #vocab = ['<pad>'] + vocab
        print("Vocab processed, {} words in total".format(len(vocab)))
        
        idx_to_word = dict(zip(range(len(vocab)), vocab))
        word_to_idx = dict(zip(vocab, range(len(vocab))))
        return idx_to_word, word_to_idx, np.array([[word_to_idx[word] for word in sequence] for sequence in data]) # this will count as all the 

In [2]:
idx_to_word, word_to_idx, x = _process_vocabs(x)

Vocab processed, 9338 words in total


In [3]:
# print(len(idx_to_word))
x = np.reshape(x, newshape=(-1,10))
np.random.shuffle(x)
x_train, x_val, x_test = x[:900], x[900:1000], x[1000:]
classes_per_set = 20 # 20 way
samples_per_class = 1 # 1 shot
indexes = {"train": 0, "val": 0, "test": 0}
datatset = {"train": x_train, "val": x_val, "test":  x_test}
use_cache = True
#np.reshape(x, newshape=(-1,10))

In [4]:
batch_size = 20
def load_data_cache(data_pack, argument=True):
        """ 
        load the data with batch, 
        """
        cached_dataset = []
        classes_idx = np.arange(data_pack.shape[0]) # length of classes
        samples_idx = np.arange(data_pack.shape[1]) # 10
        for _ in range(1000): # what is this?
            support_set_x = []
            # 20 * 20 * 1 * maxlen
            support_set_y = [] # 20 * 20 * 1
            target_x = [] # 20 * 122, 
            target_y = []# 20 * 1
            for i in range(batch_size):  
                choose_classes = np.random.choice(classes_idx, size=classes_per_set, replace=False) # choose the 20 classes
                choose_label = np.random.choice(classes_per_set, size=1) # choose which to query
                choose_samples = np.random.choice(samples_idx, size=samples_per_class + 1, replace=False) #
                x_temp = data_pack[choose_classes,:] # select 20 classes as support 
                x_temp = x_temp[:,choose_samples] # 
                y_temp = np.arange(classes_per_set)
                xx = []
                for x in x_temp:
                    xx.append(list(x[:-1]))
                support_set_x.append(xx)
                #support_set_x[i] = x_temp[:,:-1] # the samples exept the last sample
                support_set_y.append(list(y_temp)) # transpose
                target_x.append(list(x_temp[choose_label][-1])) # the query x with the chosen class
                target_y.append(y_temp[choose_label]) 
            cached_dataset.append([support_set_x, support_set_y, target_x, target_y])
        return cached_dataset # return the 1000 training epochs 

In [5]:
cached_datatset = {"train": load_data_cache(x_train),
                                    "val": load_data_cache(x_val),
                                    "test": load_data_cache(x_test)}

In [6]:
support_set_x, support_set_y, target_x, target_y = cached_datatset['train'][0]
print(len(support_set_y))

20
